In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from collections import Counter
import nltk
import psycopg2
from nltk.corpus import stopwords
import re
from itertools import chain 
from IPython.display import Image
import math
import time
import logging 

In [2]:
from gensim import models
w = models.KeyedVectors.load_word2vec_format(r"F:\Pretrained Models\GoogleNews-vectors-negative300.bin.gz", binary=True)

In [12]:
def process(array):
    text = re.sub(r'\[[0-9]*\]',' ',str(array))  #Remove Numbers
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Remove nums
    text = re.sub(r'\s+',' ',text)  #Remove extra space
    text = re.sub(r"[^a-zA-Z0-9]+",' ',text)  #Remove special characters
    text = text.lower()  #Lower case all
    text = nltk.sent_tokenize(text)  #Tokenize to sentences 
    keywords = [nltk.word_tokenize(sentence) for sentence in text]
    for i in range(len(keywords)):
        keywords[i] = [word for word in keywords[i] if word not in stopwords.words('english')]
    return keywords

#Frame pre-processing function
def prepro(frame):
    frame2 = frame
    for c in range(len(frame2)):
        keywords = process(frame2['Caption'][c])
        frame2['Caption'][c] = keywords
    return frame2

def flat(array):     # 2D array reduction
    flatten_list = [j for sub in array for j in sub] 
    flatten_list = list(chain.from_iterable(flatten_list))
    return flatten_list


# normalize() -> given an array, converts to 1/0, top int(pos) will be 1
def normalize(keys, pos =3):  
    ax = [i for i in keys]
    temp = [i for i in keys]
    temp.sort()
    temp = temp[-pos:]
    for x in temp:
        ax[keys.index(x)] = 1
    for x in range(len(ax)):
        if ax[x] != 1:
            ax[x] = 0
    return ax

# compute() => category[] to be called outside
def compute(caption,category,top =3):
    ar = []
    score = []
    
    # Code to get frequency distribution and unique keywords array
    counts = Counter(caption)
    labels, values = zip(*counts.items())
    ## sort your values in descending order
    indSort = np.argsort(values)[::-1]
    ## rearrange your data
    keywords = np.array(labels)[indSort]  # Label
    caption_freq = np.array(values)[indSort]  # Values
    
    # Detect words not in Google Dict | Put freq = 0
    for x in keywords:
        try:
            restConst = w.similarity(x,'something')
        except KeyError:
            caption_freq[np.where(keywords == x)] = 0
        
    #Google similaity function
    for x in category:
        empty = []
        for y in keywords:
            try:
                empty.append(w.similarity(x,y))
            except:
                empty.append(0)
        ar.append(empty)
    
    # Store the similarity values in dataframe
    frame = pd.DataFrame(ar, columns = keywords)
    
    #Normalize | top select
    for key in frame.columns:
        frame[key] = normalize(frame[key].tolist(),top)
    
    # Multiply with frequency
    for row in range(len(frame)):
        frame.values[row] = [i*j for i,j in zip(frame.values[row],caption_freq)]
    # Sum the values => Score
    for row in range(len(frame)):
        score.append(sum(frame.values[row]))
    
    frame['category'] = category
    frame['Scores'] = score
    return frame

def deviation(array):
    mu = max(array)
    l = len(array)
    ar = []
    for x in range(l):
        ar.append(math.sqrt((array[x]-mu)**2)/l)
    total = sum(ar)
    for x in range(l):
        ar[x] = (ar[x]/total)*100
    return ar

def sort_cat(dev,cat, thre = 2):
    final_cat = []
    for i in range(len(dev)):
        if dev[i] <=thre:
            final_cat.append(cat[i])
    return final_cat

def get_row_pscore(col_name,f1,f2):  # f1-mainframe | f2-frame
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    dev_array = f2['Deviation'].tolist()
    row_in_array.extend(dev_array)
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict
def top_category(f2,categories,thre=2):
    final_cat = [0]*len(categories)
    dev_scores = f2['Deviation'].tolist()
    rank_of_cat = 1
    for i in range(len(dev)):
        if dev_scores[i] <=thre:
            final_cat[i] = rank_of_cat 
            rank_of_cat = rank_of_cat +1
    return final_cat
def get_row_result(col_name,f1,rank_array):  # f1-mainframe
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    row_in_array.extend(rank_array)
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict

In [13]:
conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
cur = conn.cursor()

### Get IDs | <500,000 & India   |   Top 10k only

In [14]:
# cur.execute('''SELECT user_id, handle, url, followers FROM instagram_accounts WHERE (followers < 500000 and country = 'India') ORDER BY followers DESC''')
cur.execute('''SELECT  "instagram_accounts"."user_id", "instagram_accounts"."handle", "instagram_accounts"."url", "instagram_accounts"."followers" FROM "instagram_accounts" WHERE (followers < 500000 and country = 'India') ORDER BY "instagram_accounts"."followers" DESC''')
InstagramAccount = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
InstagramAccount = InstagramAccount[5000:10000]
Insta_ids = InstagramAccount['user_id'].tolist()

In [15]:
# Categories
categories = ['food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal']


# DataFrame Creation

profilescore = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
onefive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
twofive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
threefive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])

# This required to run some function

col_name = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal']


In [17]:
#Timer start
tic = time.perf_counter()
prev_tac = tic
for x in Insta_ids:
    try:
        tac = time.perf_counter()
        time_compare = tac-prev_tac
        if time_compare >= 1000:
            conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
            cur = conn.cursor()
            prev_tac = tac
        userid =x
        # Id, handle, url
        cur.execute('SELECT user_id, handle, url, followers  FROM instagram_accounts WHERE user_id = '+str(userid)+' ORDER BY user_id')
        mainframe = pd.DataFrame()
        mainframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
        # captions
        cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
        randomframe = pd.DataFrame()
        randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])
        if len(randomframe) ==0:
            raise Exception("No captions in Winkl database")
        r2 = pd.DataFrame()
        r2 = prepro(randomframe)
        # Merge
        r3 = pd.DataFrame()
        r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
        caption_array = r3['Caption'][x]
        # to 1D
        caption_array = flat(caption_array)
        # Word2vec computation
        frame = pd.DataFrame()
        frame = pd.DataFrame(compute(caption_array,categories,3))
        #Convert to Percentage
        per = frame['Scores'].tolist()
        per_sum = sum(per)
        for x in range(len(per)):
            per[x] = round((per[x]/per_sum)*100)
        frame['Percentage'] = per
        # Deviation of categories
        dev = deviation(frame['Scores'].tolist())
        frame['Deviation'] = dev
        # store profile score
        row_df_1 = get_row_pscore(col_name,mainframe,frame)
        profilescore = profilescore.append(row_df_1,ignore_index=True)

        # store onefive result
        select_15_array = top_category(frame,categories,thre=1.5)
        row_df_2 = get_row_result(col_name,mainframe,select_15_array)
        onefive = onefive.append(row_df_2,ignore_index=True)

        # store twofive result
        select_25_array = top_category(frame,categories,thre=2.5)
        row_df_3 = get_row_result(col_name,mainframe,select_25_array)
        twofive = twofive.append(row_df_3,ignore_index=True)

        # store threefive result
        select_35_array = top_category(frame,categories,thre=3.5)
        row_df_4 = get_row_result(col_name,mainframe,select_35_array)
        threefive = threefive.append(row_df_4,ignore_index=True)
    except Exception as Argument:
        
        # creating/opening a file 
        f = open("E:\Winkl Mains\DATA\Categorise model testing data\errorlogs2.txt", "a") 
        # writing in the file 
        f.write("In user_id:"+str(userid)+": "+str(Argument)+str("\n")) 
        # closing the file 
        f.close()  

display(profilescore)
display(onefive)
display(twofive)
display(threefive)

# Timer end
toc = time.perf_counter()
print(f"Ran the model in {toc - tic:0.4f} seconds")

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,5.083040,4.932058,3.497735,0.000000,4.655259,6.114746,...,5.133367,7.096125,6.567690,6.492199,8.253649,5.636638,7.876195,8.329139,5.636638,7.272270
1,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,6.224140,0.000000,3.162312,0.384206,7.128502,4.539544,...,6.809316,7.234898,8.145171,5.692162,8.251566,6.537416,7.051661,7.341293,6.744296,7.660480
2,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,5.083040,4.932058,3.497735,0.000000,4.655259,6.114746,...,5.133367,7.096125,6.567690,6.492199,8.253649,5.636638,7.876195,8.329139,5.636638,7.272270
3,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,6.224140,0.000000,3.162312,0.384206,7.128502,4.539544,...,6.809316,7.234898,8.145171,5.692162,8.251566,6.537416,7.051661,7.341293,6.744296,7.660480
4,6512,whatshappening365,51137,https://www.instagram.com/whatshappening365/,3.178552,4.431546,3.947853,4.555924,6.131380,5.679934,...,4.081445,7.674590,5.578589,5.127142,7.361341,0.000000,9.544868,9.134881,7.679197,8.554450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,59013,yoshnasbyela,40729,https://www.instagram.com/yoshnasbyela/,7.455390,0.505174,3.299927,0.000000,6.673185,6.379858,...,4.440642,5.744317,8.400554,6.290231,7.927972,5.817648,6.388006,7.650941,7.414650,8.090931
1050,43088,tetseosisters,40723,https://www.instagram.com/tetseosisters/,7.071414,3.020604,7.281456,0.000000,4.480896,7.711542,...,0.340068,5.591118,11.202240,2.080416,10.352070,4.400880,8.951790,11.552310,3.120624,9.531906
1051,8663,tiwaripreety,40717,https://www.instagram.com/tiwaripreety/,0.000000,4.752941,0.145882,0.258824,1.920000,7.190588,...,5.708235,9.520000,4.795294,5.289412,10.903529,4.804706,8.875294,11.190588,4.950588,9.223529
1052,6637,vintzdaily,40717,https://www.instagram.com/vintzdaily/,3.524668,2.686540,1.572056,0.000000,3.687763,5.835183,...,6.079826,7.330222,6.464912,6.034522,9.355321,5.508993,9.233000,9.504825,6.288225,8.236307


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6512,whatshappening365,51137,https://www.instagram.com/whatshappening365/,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,59013,yoshnasbyela,40729,https://www.instagram.com/yoshnasbyela/,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1050,43088,tetseosisters,40723,https://www.instagram.com/tetseosisters/,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,0,0
1051,8663,tiwaripreety,40717,https://www.instagram.com/tiwaripreety/,1,0,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1052,6637,vintzdaily,40717,https://www.instagram.com/vintzdaily/,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6512,whatshappening365,51137,https://www.instagram.com/whatshappening365/,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,59013,yoshnasbyela,40729,https://www.instagram.com/yoshnasbyela/,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1050,43088,tetseosisters,40723,https://www.instagram.com/tetseosisters/,0,0,0,1,0,0,...,2,0,0,3,0,0,0,0,0,0
1051,8663,tiwaripreety,40717,https://www.instagram.com/tiwaripreety/,1,0,2,3,4,0,...,0,0,0,0,0,0,0,0,0,0
1052,6637,vintzdaily,40717,https://www.instagram.com/vintzdaily/,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,0,1,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,201259,anisha_mukherjee,51144,https://www.instagram.com/anisha_mukherjee/,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,91813,amitgt_officialpage,51144,https://www.instagram.com/amitgt_officialpage/,0,1,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6512,whatshappening365,51137,https://www.instagram.com/whatshappening365/,1,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,59013,yoshnasbyela,40729,https://www.instagram.com/yoshnasbyela/,0,1,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1050,43088,tetseosisters,40723,https://www.instagram.com/tetseosisters/,0,1,0,2,0,0,...,4,0,0,5,0,0,0,0,6,0
1051,8663,tiwaripreety,40717,https://www.instagram.com/tiwaripreety/,1,0,2,3,4,0,...,0,0,0,0,0,0,0,0,0,0
1052,6637,vintzdaily,40717,https://www.instagram.com/vintzdaily/,0,1,2,3,0,0,...,0,0,0,0,0,0,0,0,0,0


Ran the model in 5165.4130 seconds


In [18]:
profilescore.to_csv(r'E:\Winkl Mains\DATA\Categorise model testing data\profile_deviation_score.csv')
onefive.to_csv(r'E:\Winkl Mains\DATA\Categorise model testing data\categorised_1_5.csv')
twofive.to_csv(r'E:\Winkl Mains\DATA\Categorise model testing data\categorised_2_5.csv')
threefive.to_csv(r'E:\Winkl Mains\DATA\Categorise model testing data\categorised_3_5.csv')

### END 

## TESTING SECTION

userid = 300226
cur.execute('SELECT user_id, handle, url, followers  FROM instagram_accounts WHERE user_id = '+str(userid)+' ORDER BY user_id')
mainframe = pd.DataFrame()
mainframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])


cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
randomframe = pd.DataFrame()
randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])
r2 = pd.DataFrame()
r2 = prepro(randomframe)
# Merge
r3 = pd.DataFrame()
r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
cont = r3['Caption'][userid]
cont = flat(cont)
frame = pd.DataFrame()
frame = pd.DataFrame(compute(cont,categories,3))
#Convert to Percentage
per = frame['Scores'].tolist()
per_sum = sum(per)
for x in range(len(per)):
    per[x] = round((per[x]/per_sum)*100)
frame['Percentage'] = per
# Deviation of categories
dev = deviation(frame['Scores'].tolist())
frame['Deviation'] = dev

# store profile score
row_df_1 = get_row_pscore(col_name,mainframe,frame)
profilescore = profilescore.append(row_df_1,ignore_index=True)

# store onefive result
select_15_array = top_category(frame,categories,thre=1.5)
row_df_2 = get_row_result(col_name,mainframe,select_15_array)
onefive = onefive.append(row_df_2,ignore_index=True)

# store twofive result
select_25_array = top_category(frame,categories,thre=2.5)
row_df_3 = get_row_result(col_name,mainframe,select_25_array)
twofive = twofive.append(row_df_3,ignore_index=True)

# store threefive result
select_35_array = top_category(frame,categories,thre=3.5)
row_df_4 = get_row_result(col_name,mainframe,select_35_array)
threefive = threefive.append(row_df_4,ignore_index=True)

display(frame)
display(mainframe)
display(profilescore)
display(onefive)
display(twofive)
display(threefive)


In [11]:
tic = time.perf_counter()
prev_tac = tic
y=0
for i in range(1000):
    try:
        tac = time.perf_counter()
        time_compare = tac-prev_tac
        if time_compare >= 60:
            print("Connecting server")
            conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
            cur = conn.cursor()
            prev_tac = tac
            i = y+1
            print(prev_tac,time_compare)
    except Exception as Argument:
        print(str(Argument)+str("\n"))